In [1]:
##importing required libraries
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import re
import pickle
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

<h1>Functions</h1>

In [2]:
#function to remove punctuation or any special characters
def removePunc(sentence): 
    processed = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    processed = re.sub(r'[.|,|)|(|\|/]',r' ',processed)
    processed = processed.strip()
    processed = processed.replace("\n"," ")
    return processed

In [3]:
def removeNonAlphabets(sentence):
    final_sent = ""
    for word in sentence.split():
        alpha_word = re.sub('[^a-z A-Z]+', ' ', word)
        final_sent += alpha_word
        final_sent += " "
    final_sent = final_sent.strip()
    return final_sent

In [4]:
def removeHTMLtags(sentence):
    tags = re.compile('<.*?>')
    cleaned = re.sub(tags, ' ', str(sentence))
    return cleaned

In [5]:
def removeStopWords(sentence):
    global re_stop_words
    return re_stop_words.sub(" ", sentence)

In [6]:
##use SnowBall Stemmer for Stemming purpose
stemmer = SnowballStemmer("english")

def stemming(sentence):
    finalSentence = ""
    for word in sentence.split():
        stemmed = stemmer.stem(word)
        finalSentence += stemmed
        finalSentence += " "
    finalSentence = finalSentence.strip()
    return finalSentence

In [7]:
def clean_text(text_data):
    text_data = text_data.apply(removeHTMLtags)
    text_data = text_data.apply(removePunc)
    text_data = text_data.apply(removeNonAlphabets)
    text_data = text_data.apply(removeStopWords)
    text_data = text_data.apply(stemming)
    return text_data

In [8]:
def read_database(filename):
    '''
        Function reads a SQLite Database File and converts it to a DataFrame
    '''
    # Creating SQLite Engine to import DataFrame as DataBase
    engine = create_engine('sqlite:///DisasterResponse.db')
    data = pd.read_sql_table(table_name = 'InsertTableName', con = engine)
    return data

In [9]:
def vector_generator(train_df, test_df, analyzer_, stop_words_, max_features_, labels_to_drop):
    '''
        Function takes in training and testing data and generates and returns feature vectors
    '''
    vectorizer = TfidfVectorizer(analyzer = analyzer_ , stop_words = stop_words_, max_features = max_features_)

    train_text = train_df["message"]
    test_text = test_df["message"]

    vectorizer.fit(train_text)
    vectorizer.fit(test_text)

    x_train = vectorizer.fit_transform(train_text)
    y_train = train_df.drop(labels = labels_to_drop, axis=1)

    x_test = vectorizer.fit_transform(test_text)
    y_test = test_df.drop(labels =  labels_to_drop, axis=1)
    
    return x_train, y_train, x_test, y_test

In [10]:
def generate_results(y_test_array, predictions, fields):
    '''
        Function accepts test data output, predictions from model, and fields and returns classification report 
    '''
    return classification_report(y_test_array, np.array([x[:] for x in predictions]), target_names = fields)

In [11]:
def best_estimator(training_x, training_y, model_parameters, cross_validation_count):
    '''
        Function takes in training data and model_parameters to be tested, 
        returns bests parameters found using GridSearch
    '''
    # defining parameter range
    researched_grid = GridSearchCV(RandomForestClassifier(), model_parameters, cv=cross_validation_count, scoring='accuracy', return_train_score=False, verbose=1)

      # fitting the model for grid search
    grid_research=researched_grid.fit(training_x, training_y)

    # getting the best estimator
    best_model = grid_research.best_estimator_ 
    
    return best_model

<h1>Deployment using Random Forest CLassifier and TF-IDF</h1>

In [12]:
if __name__ == "__main__":
    # Downloading stopwords
    nltk.download('stopwords')
    stop_words = set(stopwords.words('english'))
    re_stop_words = re.compile(r"\b(" + "|".join(stop_words) + ")\\W", re.I)##compiling into regex to remove them

    ##use SnowBall Stemmer for Stemming purpose
    stemmer = SnowballStemmer("english")
    
    df = read_database('sqlite:///DisasterResponse.db')
    print(df.head())
    
    df.drop_duplicates(subset = "id", inplace=True)
    
    df["message"] = clean_text(df["message"])
    # replaces anomalous values (2) in data with 1
    df["related"] = df["related"].map(lambda a: 1 if a > 1 else 0)
    df.to_csv("/Cleaned.csv")
    print(df.head())
    
    train, test = train_test_split(df, random_state=42, test_size=0.33, shuffle=True)

    # Feature Vectors are generated
    x_train, y_train, x_test, y_test = vector_generator(train, test, "word", "english", 500, ['message','id','original','genre'])

    #RandomForestClassifier is used to build model
    clf2 = MultiOutputClassifier(RandomForestClassifier()).fit(x_train, y_train)
    predicted = clf2.predict(x_test)
    
    print(generate_results(y_test.iloc[:, :].values, predicted, y_test.columns))

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/zeeshan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


   id                                            message  \
0   2  Weather update - a cold front from Cuba that c...   
1   7            Is the Hurricane over or is it not over   
2   8                    Looking for someone but no name   
3   9  UN reports Leogane 80-90 destroyed. Only Hospi...   
4  12  says: west side of Haiti, rest of the country ...   

                                            original   genre  related  \
0  Un front froid se retrouve sur Cuba ce matin. ...  direct        1   
1                 Cyclone nan fini osinon li pa fini  direct        1   
2  Patnm, di Maryani relem pou li banm nouvel li ...  direct        1   
3  UN reports Leogane 80-90 destroyed. Only Hospi...  direct        1   
4  facade ouest d Haiti et le reste du pays aujou...  direct        1   

   request  offer  aid_related  medical_help  medical_products  ...  \
0        0      0            0             0                 0  ...   
1        0      0            1             0              

/home/zeeshan/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/zeeshan/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/zeeshan/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/zeeshan/anaconda3/lib/python3.

# Using bag of words feature extractor and random forest classifier(best performance on targeted data)

In [13]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features=500, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
X_train = vectorizer.fit_transform(train["message"]).toarray()
X_test = vectorizer.fit_transform(test["message"]).toarray()

In [14]:
X_train.shape

(17540, 500)

In [15]:
Y_train = train.drop(["id", "message", "original", "genre"], axis = 1)

In [16]:
Y_test = test.drop(["id", "message", "original", "genre"], axis = 1)

In [17]:
Y_train.shape

(17540, 36)

In [18]:
# RandomForestClassifier is used to build model
clf3 = MultiOutputClassifier(RandomForestClassifier()).fit(X_train, Y_train)
predicted = clf3.predict(X_test)


In [19]:
print(generate_results(Y_test.iloc[:, :].values, predicted, Y_test.columns))

                        precision    recall  f1-score   support

               related       0.05      0.04      0.05        68
               request       0.42      0.09      0.15      1466
                 offer       0.00      0.00      0.00        41
           aid_related       0.54      0.36      0.43      3549
          medical_help       0.38      0.06      0.11       688
      medical_products       0.46      0.17      0.25       411
     search_and_rescue       0.00      0.00      0.00       245
              security       0.00      0.00      0.00       170
              military       0.06      0.01      0.02       286
           child_alone       0.00      0.00      0.00         0
                 water       0.38      0.02      0.04       543
                  food       0.21      0.16      0.18       939
               shelter       0.33      0.05      0.09       762
              clothing       0.03      0.01      0.01       123
                 money       0.15      

/home/zeeshan/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/zeeshan/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/zeeshan/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/zeeshan/anaconda3/lib/python3.

# Using bag of words feature extractor and Kneighbor classifier

In [20]:
# RandomForestClassifier is used to build model
from sklearn.neighbors import KNeighborsClassifier
clf4 = MultiOutputClassifier(KNeighborsClassifier()).fit(X_train, Y_train)
predicted = clf4.predict(X_test)

In [21]:
print(generate_results(Y_test.iloc[:, :].values, predicted, Y_test.columns))

                        precision    recall  f1-score   support

               related       0.00      0.00      0.00        68
               request       0.33      0.02      0.04      1466
                 offer       0.00      0.00      0.00        41
           aid_related       0.60      0.09      0.15      3549
          medical_help       0.82      0.01      0.03       688
      medical_products       0.50      0.00      0.01       411
     search_and_rescue       0.00      0.00      0.00       245
              security       0.00      0.00      0.00       170
              military       0.00      0.00      0.00       286
           child_alone       0.00      0.00      0.00         0
                 water       0.60      0.01      0.01       543
                  food       0.12      0.01      0.02       939
               shelter       0.50      0.00      0.01       762
              clothing       0.00      0.00      0.00       123
                 money       0.00      

/home/zeeshan/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/zeeshan/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/zeeshan/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/zeeshan/anaconda3/lib/python3.

# For hyperparameters fine-tuning using GridCV and deploying Best Estimator

In [ ]:
if __name__ == "__main__":
        

    # Combinations of parameters for GridSearch
    # For faster run use these parameters
    researched_parameters = {'bootstrap': [True],
      'criterion':["gini", "entropy"],
     'max_depth': [80, None],
     'min_samples_leaf': [1, 5],
     'min_samples_split': [2, 12],
     'n_estimators': [50, 100, 200]
     }

    '''
    #  For exhaustive analysis and determination of best parameter use these:
    #  Takes longer time
    param_grid = {
        'bootstrap': [True],
        'criterion':["gini", "entropy"]
        'max_depth': [80, 90, 100, None],
        'min_samples_leaf': [1, 4, 5],
        'min_samples_split': [2, 10, 12],
        'n_estimators': [50, 100, 500, 200]
    }
    '''

    # getting the best estimator
    best_clf_re = best_estimator(x_train, y_train, researched_parameters, 5)
    best_predictions_re = best_clf_re.predict(x_test)

    print(generate_results(y_test.iloc[:, :].values, best_predictions_re, y_test.columns))
    
    # saving the best model as a pickle file
    pickle.dump(best_clf_re, open('Classification_model.pkl','wb'))

In [23]:
pickle.dump(clf2, open('Classification_model.pkl','wb'))